In [54]:
import os
import openai
import datetime
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma


from dotenv import load_dotenv, find_dotenv

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma


_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [55]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core import VectorStoreIndex

In [56]:
BASE_PATH = "C:\\Users\\vital\\iCloudDrive\\iCloud~md~obsidian\\new_vault\\"


documents = ObsidianReader(
    BASE_PATH
).load_data()  # Returns list of documents

In [57]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

558 

<class 'llama_index.core.schema.Document'>
Doc ID: 1b961cac-79bd-4ee2-8733-8af60844aa0a
Text: References [[(MOC) How to get rich]]


In [58]:
from llama_index.core  import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [59]:
index = VectorStoreIndex.from_documents(
    documents
)  # Initialize index with documents

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x000002167E3EBEC0>
Traceback (most recent call last):
  File "C:\Users\vital\streamlit_suite\.venv\Lib\site-packages\openai\_base_client.py", line 714, in __del__
    self.close()
  File "C:\Users\vital\streamlit_suite\.venv\Lib\site-packages\httpx\_client.py", line 1264, in close
    def close(self) -> None:

KeyboardInterrupt: 


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "

In [60]:
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI as LIOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

C:\Users\vital\AppData\Local\Temp\ipykernel_13688\855792854.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [61]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(k=10)
res = query_engine.query("How to structure a writing routine?")
res.response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"To structure a writing routine effectively, you should consider the following steps:\n\n1. **Set Clear Goals**: Define what you aim to achieve with your writing routine, whether it's to improve your skills, share knowledge, or build a portfolio.\n\n2. **Establish a Schedule**: Allocate specific time slots for writing in your daily or weekly calendar to ensure consistency and productivity.\n\n3. **Create a Dedicated Workspace**: Designate a comfortable and inspiring area where you can focus solely on your writing tasks.\n\n4. **Plan Your Topics**: Outline the subjects or themes you want to cover in your writing to maintain a sense of direction and purpose.\n\n5. **Practice Active Summarization**: Summarize key insights from your readings in your own words to enhance comprehension and retention.\n\n6. **Implement Tagging and Categorization**: Organize your notes and ideas using tags or categories to facilitate easy retrieval and connection of information.\n\n7. **Schedule Review Session

In [64]:
import os
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [72]:
from llama_index.llms.openai import OpenAI


index = build_sentence_window_index(
    [document],
    llm=llm,
    save_dir="./sentence_index",
)


C:\Users\vital\AppData\Local\Temp\ipykernel_13688\678983494.py:22: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [73]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)


In [74]:
import numpy as np
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

from trulens_eval.feedback import Groundedness

def get_prebuilt_trulens_recorder(query_engine, app_id):
    openai = OpenAI()

    qa_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
        .on_input_output()
    )

    qs_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(TruLlama.select_source_nodes().node.text)
        .aggregate(np.mean)
    )

    grounded = Groundedness(groundedness_provider=openai)

    groundedness = (
        Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
            .on(TruLlama.select_source_nodes().node.text)
            .on_output()
            .aggregate(grounded.grounded_statements_aggregator)
    )

    feedbacks = [qa_relevance, qs_relevance, groundedness]
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder

In [82]:
eval_questions = [
    "What are the most effective strategies for building long-term wealth?",
    "How does investment diversification contribute to wealth accumulation?",
    "Can you explain the importance of compound interest in wealth building?",
    "What role does entrepreneurship play in achieving financial independence?",
    "How crucial is financial literacy in the process of getting rich?",
    "Describe the risks and benefits of real estate investment as a wealth-building strategy.",
    "How can one balance risk and reward in stock market investments?",
    "What are common financial mistakes people should avoid when trying to get rich?",
    "How can setting financial goals contribute to wealth accumulation?",
    "Explain the impact of personal savings rate on the path to becoming rich.",
    "What are the key elements of a compelling narrative?",
    "How can a writer effectively develop characters in a story?",
    "Describe the process of outlining a story before writing.",
    "How important is the setting in a story, and how can it be effectively described?",
    "What techniques can writers use to maintain reader interest throughout a piece?",
    "Discuss the role of conflict in storytelling and how it can be used effectively.",
    "What are some common pitfalls in writing dialogue, and how can they be avoided?",
    "How can writers improve their writing style and voice?",
    "Explain the revision process and its importance in writing.",
    "How can feedback from others be incorporated into the writing process?"
]


In [76]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [83]:
from trulens_eval import Tru

Tru().reset_database()

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:alembic.runtime.migration:Running upgrade  -> 1, first revision
Running upgrade  -> 1, first revision
Running upgrade  -> 1, first revision
INFO:trulens_eval.database.sqlalchemy_db:Your database does not need migration.
Your database does not need migration.
Your database does not need migration.


In [84]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1",
)

C:\Users\vital\AppData\Local\Temp\ipykernel_13688\678983494.py:22: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [85]:
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

In [86]:
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added app sentence window engine 1
✅ added app sentence window engine 1
✅ added app sentence window engi

In [ ]:
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)
Tru().run_dashboard()

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.85s/it]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added record record_hash_0b83030a7deaabc3cc6a22085240a368
✅ added record record_hash_0b83030a7deaabc3cc6a22085240a368
✅ added record record_hash_0b83030a7deaabc3cc6a22085240a368
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:alembic.runtime.migration:Context impl 

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added record record_hash_6319ea84d539780284708ef7e18cfe86
✅ added record record_hash_6319ea84d539780284708ef7e18cfe86
✅ added record record_hash_6319ea84d539780284708ef7e18cfe86
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
Starting dashboard ...
Config file already e

ModuleNotFoundError: 
ipython,ipywidgets packages are required for using trulens_eval in a notebook.
You should be able to install them with pip:

    ```bash
    pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"
    ```


INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result Answer Relevance DONE feedback_result_hash_ed0c6bdd7cbe699656ca7a7ad4903136
✅ feedback result Answer Relevance DONE feedback_result_hash_ed0c6bdd7cbe699656ca7a7ad4903136
✅ feedback result Answer Relevance DONE feedback_result_hash_ed0c6bdd7cbe699656ca7a7ad4903136
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result Context Relevance DONE feedback_result_hash_cd287beee42145abdd908c79b4d83289
✅ feedback result Context Relevance DONE feedback_result_

In [ ]:
sentence_index_3 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)
Tru().run_dashboard()